# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name: Vijay Sharma

Student ID: 16BD02047

Email: vijay11fce@gmail.com

In [ ]:
import numpy as np
import pandas as pd
import sklearn.neighbors
import sklearn.ensemble as ensemble
import math
import matplotlib.pyplot as plt
%matplotlib inline

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [ ]:
data = pd.read_csv("2008.csv.bz2")
data.describe()

In [ ]:
1.Какая из причин отмены рейса (CancellationCode) была самой частой? (расшифровки кодов можно найти в описании данных)

In [ ]:
df = pd.read_csv("2008.csv.bz2")
print(df.head())

In [ ]:
CC = df['CancellationCode']
CC.value_counts().plot(kind='bar')

In [ ]:
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.

In [ ]:
print("Max: ", df.Distance.max())
print("Min: ", df.Distance.min())
print("Mean: ", df.Distance.mean())

In [ ]:
3. Не выглядит ли подозрительным минимальное пройденное расстояние. В какие дни и на каких рейсах оно было.Какое расстояние было пройдено этими же рейсами в другие дни.

In [ ]:
df[df.Distance == df.Distance.min()]


In [ ]:
df[DF.FlightNum == 4988]['Distance'].values

In [ ]:
df[df.FlightNum == 5572]['Distance'].values

In [ ]:
Из какого аэропорта было произведено больше всего вылетов. В каком городе он находится.

In [ ]:
df["Origin"].value_counts().index[0]

In [ ]:
Найдите для каждого аэропорта среднее время полета (AirTime) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя.

In [ ]:
df.groupby("Origin")['AirTime'].mean()

In [ ]:
print("Max mean air time: ", DF.groupby("Origin")['AirTime'].mean().argmax())

In [ ]:
Найдите аэропорт, у которого наибольшая доля задержанных (DepDelay > 0) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию filter после groupby).

In [ ]:
DFGB = DF.groupby("Origin")['DepDelay']
DFGB_Filtered = DF[DF.DepDelay > 0].groupby("Origin")['DepDelay']
(DFGB_Filtered.count() / DFGB.count() * (DFGB.count() >= 1000)).argmax()

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [ ]:
data = pd.read_csv('amazon.csv')
data.head()

In [ ]:
data.shape

In [ ]:
# доля положительных примеров
data.ACTION.mean()

In [ ]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)